# 뉴스의 감성분석과 주가등락
#### 어떤 뉴스가 시가총액 TOP20 종목의 주가등락에 영향을 주는가

  1. 3월 19일을 기점으로, 투자자들이 새로운 뉴스에 **더 민감하게** 반응할 것이다
  2. 뉴스와 투자자별 순매수비율 정보를 모두 이용하면 **정보를 선별**하여 **주가를 예측**할 수 있다. 

## 데이터 수집 및 전처리

### [수집] 1. 네이버 금융 종목뉴스 크롤링

In [ ]:
def NaverFinance_news(i):
  
    url = "https://finance.naver.com/item/news.nhn?code=%s" %top20['종목코드'][i]

    driver = webdriver.Chrome("./chromedriver")
    driver.get(url)
    time.sleep(1)

    print("------ %s(%s) crawling ------" %(top20['종목명'][i], top20['종목코드'][i]))

    end = datetime.datetime.strptime(END_DATE, "%Y-%m-%d")
    print("...START")
    page = 2
    next_ = 0

    news_title = []
    news_date = []

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    driver.switch_to.frame('news')

    while (end >= datetime.datetime.strptime(START_DATE, "%Y-%m-%d")):

        hiddennews = driver.find_elements_by_css_selector("tbody td > div > a")

        for hn in hiddennews:
            hn.click()

        title = [t.text for t in driver.find_elements_by_css_selector("td.title")]
        date = [datetime.datetime.strptime(d.text, '%Y.%m.%d %H:%M') for d in driver.find_elements_by_css_selector("td.date")]
        end = date[-1]

        for tt in date:
            if tt <= datetime.datetime.strptime(END_DATE, "%Y-%m-%d"):
                news_title.append(title[date.index(tt)])
                news_date.append(tt)

    #page: nth(2) 2페이지 > nth(4) 3페이지 ~ nth(11) 10페이지 > nth(12) = 다음 
    #      > nth(4) 12페이지 ~ nth(12) 20페이지 > nth(13) = 다음
    #      > [반복]
        if (page == 2 and next_ == 0):
            page_more = driver.find_element_by_css_selector("body > div > table.Nnavi > tbody > tr > td:nth-child(%d) > a" %(page))
            page = 3

        elif (page == 12 and next_ == 0) or (page == 13 and next_ > 0):
            page_more = driver.find_element_by_css_selector("body > div > table.Nnavi > tbody > tr > td.pgR > a")
            next_ += 1
            page = 3

        else:
            page_more = driver.find_element_by_css_selector("body > div > table.Nnavi > tbody > tr > td:nth-child(%d) > a" %(page))

        page_more.click()
        page += 1



    print("total:", len(news_title))
    print("last date:", news_date[-1])

    print("Saving")
    df = pd.DataFrame({"Date": news_date, "Title": news_title})
    df.to_csv("%s_NaverFinance.csv" %top20['종목명'][i])
    
    print("DONE...")
    driver.close()

In [ ]:
for i in range(len(top20)):
    NaverFinance_news(i)

### [수집] 2. 네이버 뉴스 크롤링 추가

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

title_text=[]
date_text =[]
for page in range(0,68):
#    url="https://search.naver.com/search.naver?&where=news&query=삼성전자&sm=tab_pge&sort=1&photo=3&field=1&reporter_article=&pd=3&ds=2019.12.01&de=2020.07.02&docid=&nso=so:dd,p:from20191201to20200702,a:t&mynews=1&start="+str(page)+"1&refresh_start=0"
    url = "https://search.naver.com/search.naver?&where=news&query=현대차&sm=tab_pge&sort=1&photo=3&field=1&reporter_article=&pd=3&ds=2019.12.01&de=2020.02.27&docid=&nso=so:dd,p:from20191201to20200227,a:t&mynews=1&start="+str(page)+"1&refresh_start=0"
    raw = requests.get(url)
    soup = BeautifulSoup(raw.text, "html.parser")
    atags = soup.select('.news_tit')
    for atag in atags:
        title_text.append(atag.text) #제목
        
    date_lists = soup.select('div.info_group > span.info')
    for date_list in date_lists:
        date_text.append(date_list.text)
    print(f'{page}_Completed')
#    print()
    
only_date_text = date_text[1::2]        
result = {"date" : only_date_text, "title" : title_text}
df1 = pd.DataFrame(result)

df1.to_csv("Hyundai_191201_200227_sorted.csv")

### [전처리] 1. 뉴스 제목 감성분석

In [ ]:
from google.colab import files

#### 한글 → 영어 번역

In [ ]:
for t in range(len(top20['종목명'])):
  data = pd.read_csv("/content/gdrive/Shared drives/2020-2 경제학특강4/NaverFinance/%s_NaverFinance.csv" %top20['종목명'][t], encoding = 'UTF-8').drop(columns = 'Unnamed: 0')

  from googletrans import Translator

  eng = []
  
  for i in range(len(data.Title)):
    tl = Translator()
    result = tl.translate(data['Title'][i], dest = "en", src = 'ko').text
    eng.append(result)

  data['EngTitle'] = eng


  data.to_csv("%s_Trans.csv" %top20['종목명'][t], index = False)
  files.download("%s_Trans.csv" %top20['종목명'][t])

#### 감성사전 이용하기

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
pos = pd.read_excel("/content/drive/Shared drives/2020-2 경제학특강4/뉴스 제목 번역/Word list/Positive.xlsx")
positive = list(pos.words.apply(lambda x: x.lower()))

neg = pd.read_excel("/content/drive/Shared drives/2020-2 경제학특강4/뉴스 제목 번역/Word list/Negative.xlsx")
negative = list(neg.words.apply(lambda x: x.lower()))

neu = pd.read_excel("/content/drive/Shared drives/2020-2 경제학특강4/뉴스 제목 번역/Word list/Uncertainty.xlsx")
neutral = list(neu.words.apply(lambda x: x.lower()))

posneg = positive + negative + neutral

In [ ]:
def data_text_cleaning(data):
 
    # 영문자 이외 문자는 공백으로 변환
    only_english = re.sub('[^a-zA-Z]', ' ', data)
 
    # 소문자 변환
    no_capitals = only_english.lower().split()
 
    # 불용어 제거
    stops = set(stopwords.words('english'))
    no_stops = [word for word in no_capitals if not word in stops]
 
    # 어간 추출
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmer_words = [stemmer.stem(word) for word in no_stops]
 
    return stemmer_words

In [ ]:
# Token 기준으로 긍/부/중 단어 count 후 라벨링

label = []
title_label = {"title": [], "token": [], 'label': label}

for t in data.EngTitle:
    pos_c = 0
    neg_c = 0
    neu_c = 0
#     t = tl.translate(t, dest = "en").text
    t = re.sub('[^a-zA-Z#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', ' ', t)
    title_label['title'].append(t)
    
    t = data_text_cleaning(t)
    title_label['token'].append(t)
    t = ' '.join(t)
    
    for i in range(len(posneg)):
        if i < len(positive):
            if t.find(posneg[i]) != -1:
                pos_c += 1
        elif i >= len(positive):
            if t.find(posneg[i]) != -1:
                neg_c += 1
        elif i >= len(positive+negative):
            if t.find(posneg[i]) != -1:
                neu_c += 1
        
    if pos_c > neg_c:
        label.append(1)
    elif pos_c < neg_c:
        label.append(-1)
    elif max(pos_c, neg_c, neu_c) == neu_c or pos_c == neg_c:
        label.append(0)

title_label['label'] = label

In [ ]:
data['Title_token'] = title_label['token']
data['posneg'] = title_label['label']

# 단어 수가 너무 적은 뉴스 기사는 drop
for i in range(len(data)):
    if len(data.Title_token[i]) >= 0 and len(data.Title_token[i]) <= 3:
        data.drop([i], inplace = True)

#### Vader 감성분석

In [ ]:
def get_sentiment(title, threshold):
    final_sentiment = 0
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(title)
    # print(scores)
    
    compound_score = scores['compound']
    if compound_score >= threshold: 
        final_sentiment = 1
    elif compound_score <= -threshold:
        final_sentiment = -1
    else: final_sentiment = 0
#     final_sentiment = 1 if compound_score >= threshold else 0
    return final_sentiment

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

data['vader_pred'] = data.EngTitle.apply(lambda x: get_sentiment(x, 0.01))

### [전처리] 2. 일별데이터 변환

- 장 마감 이후에는 다음날 데이터 처리
- 다음날 주가 데이터가 없거나 공휴일일 경우 drop

### [전처리] 3. 뉴스의 영향력 평가

#### ATR_14

In [ ]:
df = data
df['HL'] = df.High-df.Low; df['CH'] = df.Close.shift(1)-df.High; df['CL'] = df.Close.shift(1)-df.Low
df['absCH'] = np.abs(df['CH']); df['absCL'] = np.abs(df['CL'])
df1 = df.iloc[:,[6,9,10]]
df2 = pd.concat([data,df1.max(axis=1)],axis=1)
df2.columns = ['Open','High','Low','Close','Volume','Change','HL','CH','CL','ACH','ACL','TR']
df2['ATR_14'] = df2.TR.rolling(window=14).mean()
df3 = df2.iloc[:,[0,1,2,3,4,5,11,12]]
df3.head(20)

## 최종 데이터

In [ ]:
for tick in top20['종목명']:
  cls = pd.read_csv("/content/drive/Shareddrives/2020-2 경제학특강4/금융데이터/원본다운로드/%s_주가데이터.csv" %tick)[['Date', 'Close', 'Volume', 'Change', 'TR', 'ATR_14']]
  posneg = pd.read_csv("/content/drive/Shareddrives/2020-2 경제학특강4/긍정 부정 중립 Label/Label 원본/%s_label.csv" %tick)

  posneg.drop(columns = ['EngTitle', 'vader_pred'], inplace = True)
  new = []
  for date in posneg.Date:
    date = date.split(" ")[0]
    new.append(date)
  posneg['Date'] = new
  count = posneg.groupby(by = 'Date', as_index = False).count()[['Date', 'compound']].rename(columns = {'compound' : "count"})
  posneg = posneg.merge(count, how = 'left', on = 'Date')
  posneg = posneg.groupby(by = "Date", as_index = False).mean()

  invest = pd.read_csv("/content/drive/Shareddrives/2020-2 경제학특강4/투자자별 거래량/투자자별 거래량_최종/%s_투자자매매동향_기준일편집.csv" %tick)
  invest = invest[['날짜', '기관 매수', '기관 매도', '외국인 매수', '외국인 매도', '개인 매수', '개인 매도', '개인매수비율(%)',
       '개인매수비율증가(%p)', '개인매도비율(%)', '개인매도비율증가(%p)', '개인 순매수비율', '기관 순매수비율',
       '외국인 순매수비율']]
  invest = invest.rename(columns = {'날짜': "Date"})

  df = cls.merge(posneg, on = 'Date', how = 'left').merge(invest, on = 'Date', how = 'left')
  df.to_csv("/content/drive/Shareddrives/2020-2 경제학특강4/합체 데이터/%s_합체.csv" %tick)

## 데이터 분석, 예측

#### 1. 감성사전을 이용한 학습
- [솜씨좋은장씨](https://somjang.tistory.com/entry/Keras기사-제목을-가지고-긍정-부정-중립-분류하는-모델-만들어보기)

In [ ]:
def acc_loss_plot(tick, hist):
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()
    
    loss_ax.plot(hist.history['loss'], 'r--', label = 'train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label = 'val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc = 'upper left')
    
    acc_ax.plot(hist.history['accuracy'], 'b--', label = 'train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'b', label = 'val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc = 'upper right')
    
    plt.title("%s Label (%.2f%%)" %(tick, model.evaluate(X_test, y_test)[1] * 100))
    plt.savefig("/content/drive/Shareddrives/2020-2 경제학특강4/%s Label.png" %tick)
    plt.show()

In [ ]:
# 2020년 8월 31일 기준으로 train, test 나누기
train_data = data[data.Date < '2020-09-01']
test_data = data[data.Date >= '2020-09-01']

# 0(중립)인 기사가 굉장히 많음을 볼 수 있음
print(train_data.groupby('posneg').size().reset_index(name='count')) 
print(test_data.groupby('posneg').size().reset_index(name='count'))

In [ ]:
X_train = train_data.Title_token
X_test = test_data.Title_token

- 토큰화 정수 인코딩

In [ ]:
from keras.preprocessing.text import Tokenizer

max_words = 35000 # ?
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

- One-hot encoding

In [ ]:
import numpy as np

y_train = []
y_test = []

for i in range(len(train_data['posneg'])):
    if train_data['posneg'].iloc[i] == 1:
        y_train.append([0, 0, 1])
    elif train_data['posneg'].iloc[i] == 0:
        y_train.append([0, 1, 0])
    elif train_data['posneg'].iloc[i] == -1:
        y_train.append([1, 0, 0])

        
for i in range(len(test_data['posneg'])):
    if test_data['posneg'].iloc[i] == 1:
        y_test.append([0, 0, 1])
    elif test_data['posneg'].iloc[i] == 0:
        y_test.append([0, 1, 0])
    elif test_data['posneg'].iloc[i] == -1:
        y_test.append([1, 0, 0])

y_train = np.array(y_train)
y_test = np.array(y_test)

- Model 쌓기

In [ ]:
from keras.layers import Embedding, Dense, LSTM, Bidirectional, SimpleRNN
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences 
import tensorflow as tf

max_len = 20 # 전체 데이터의 길이를 20로 맞춘다 
X_train = pad_sequences(X_train, maxlen=max_len) 
X_test = pad_sequences(X_test, maxlen=max_len)  

In [ ]:
tf.keras.backend.clear_session()

model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))

model.add(Dense(3, activation = 'softmax'))

# RMSProp은 이전 누적치와 현재 gradient의 좌표별 제곱의 가중치 평균 (최근값과 과거값에서 어떤것에 중점을 줄지 정하여 계산)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = model.fit(X_train, y_train, epochs = 10, batch_size = 10, validation_split = 0.1)

In [ ]:
key = ['Samsung Electronics', 'SK Hynix', 'Samsung Biologics', 'Naver', 'LG Chem', "Samsung Electronics Woo", 'Hyundai Motor', 'Celltrion', 'Samsung SDI', 'Kakao', 'LG health', 
       'Hyundai Mobis', 'Samsung C&T', 'KIA Motors', 'Posco', 'NCsoft', 'SK Telecom', 'KB', 'Shinhan', 'LG Electronics']

In [ ]:
acc_loss_plot(key[0], history)

#### 2. 유의미한 변수 찾기

In [ ]:
import pandas as pd
import numpy as np
import sklearn 
import time
import matplotlib.pyplot as plt
import seaborn as sns;sns.set()
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
top20 = pd.read_csv("/content/drive/Shared drives/2020-2 경제학특강4/financedata.csv").head(20)
top20

In [ ]:

RF_result = {}
record = pd.DataFrame()
path = "/content/drive/Shareddrives/2020-2 경제학특강4/"
# for t in top20['종목명'][0:5]:
data = pd.read_csv(path+"합체 데이터/%s_합체.csv" %t).drop(columns = ['Unnamed: 0'])
df = data.copy()
# df['Ret'] = df['Close'].pct_change()
RFdata = df.copy()
for w in range(-3,4):
  df['com('+str(w)+')'] = df.compound.shift(w)
  df['posneg('+str(w)+')'] = df.posneg.shift(w)
for k in range(-3,4):
  df['개인순('+str(k)+')'] = df['개인 순매수비율'].shift(k)-df['개인 순매수비율'].shift(k+1)
for v in range(-3,4):
  df['기관순('+str(v)+')'] = df['기관 순매수비율'].shift(v)-df['기관 순매수비율'].shift(v+1)
for z in range(-3,4):
  df['외국인순('+str(z)+')'] = df['외국인 순매수비율'].shift(z)-df['외국인 순매수비율'].shift(z+1)
df0 = df.iloc[:,3]
df1 = df.iloc[:,25:39]; df11 = pd.concat([df0,df1], axis=1).dropna()
df2 = df.iloc[:,39:46]; df21 = pd.concat([df0,df2], axis=1).dropna()
df3 = df.iloc[:,46:53]; df31 = pd.concat([df0,df3], axis=1).dropna()
df4 = df.iloc[:,53:60]; df41 = pd.concat([df0,df4], axis=1).dropna()

X = df11.iloc[:,1:]
y = np.sign(df11['Change']) 

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.1)

clf_rf = RandomForestClassifier()  # instantiation
clf_rf.fit(Xtrain, ytrain )   # fitting
feature_imp = clf_rf.feature_importances_

feature_imp_series = pd.Series(feature_imp, index=Xtrain.columns)
feature_imp_series.sort_values(axis=0, ascending=False, inplace=True)
name1 = list(feature_imp_series.index)
rate1 = list(feature_imp_series.values)

X2 = df21.iloc[:,1:]
y2 = np.sign(df21['Change']) 

Xtrain2, Xtest2, ytrain2, ytest2 = train_test_split(X2, y2,test_size = 0.1)

clf_rf2 = RandomForestClassifier()  # instantiation
clf_rf2.fit(Xtrain2, ytrain2 )   # fitting
feature_imp2 = clf_rf2.feature_importances_

feature_imp_series2 = pd.Series(feature_imp2, index=Xtrain2.columns)
feature_imp_series2.sort_values(axis=0, ascending=False, inplace=True)
name2 = list(feature_imp_series2.index)
rate2 = list(feature_imp_series2.values)

X3 = df31.iloc[:,1:]
y3 = np.sign(df31['Change']) 

Xtrain3, Xtest3, ytrain3, ytest3 = train_test_split(X3, y3, test_size = 0.1)

clf_rf3 = RandomForestClassifier()  # instantiation
clf_rf3.fit(Xtrain3, ytrain3 )   # fitting
feature_imp3 = clf_rf3.feature_importances_

feature_imp_series3 = pd.Series(feature_imp3, index=Xtrain3.columns)
feature_imp_series3.sort_values(axis=0, ascending=False, inplace=True)
name3 = list(feature_imp_series3.index)
rate3 = list(feature_imp_series3.values)

X4 = df41.iloc[:,1:]
y4 = np.sign(df41['Change']) 

Xtrain4, Xtest4, ytrain4, ytest4 = train_test_split(X4, y4, test_size = 0.1)

clf_rf4 = RandomForestClassifier()  # instantiation
clf_rf4.fit(Xtrain4, ytrain4 )   # fitting
feature_imp4 = clf_rf4.feature_importances_

feature_imp_series4 = pd.Series(feature_imp4, index=Xtrain4.columns)
feature_imp_series4.sort_values(axis=0, ascending=False, inplace=True)
name4 = list(feature_imp_series4.index)
rate4 = list(feature_imp_series4.values)

print(f'{name1[0]},{name2[0]},{name3[0]},{name4[0]} : {t}_completed')
r_list = [name1[0],name2[0],name3[0], name4[0]]
record.insert(loc =0, column = t, value = r_list)

#### Random Forest 주가등락 예측

In [ ]:
RFdata[name1[0]] = df[name1[0]]
RFdata[name2[0]] = df[name2[0]]
RFdata[name3[0]] = df[name3[0]]
RFdata[name4[0]] = df[name4[0]]
RFdata['TR/ATR'] = df['TR']/df['ATR_14']
RF_copy1 = RFdata.iloc[:,[3,11]]
RF_copy2 = RFdata.iloc[:,25:]
RF_df = pd.concat([RF_copy1,RF_copy2], axis = 1).dropna()
# RF_df

X_rf = RF_df.iloc[:,1:]
y_rf = np.sign(RF_df['Change']) 

Xtrain_rf, Xtest_rf, ytrain_rf, ytest_rf = train_test_split(X_rf, y_rf, test_size = 0.1)

clf_rfc_rf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
clf_rfc_rf.fit(Xtrain_rf, ytrain_rf)
yhat_rfc_rf = clf_rfc_rf.predict(Xtest_rf)
acc = accuracy_score(yhat_rfc_rf, ytest_rf)
print(f'{t}_RF_accuracy : {acc}\n')

x_corr = X_rf.corrwith(y_rf)
x_corr.sort_values(axis = 0, ascending = False)
display(pd.DataFrame({t: x_corr.sort_values(axis = 0, ascending = False)}))

print("linear corr: ", )
RF_result[t] = acc

feature_imp_rf = clf_rfc_rf.feature_importances_
feature_imp_series_rf = pd.Series(feature_imp_rf, index=Xtrain_rf.columns)
feature_imp_series_rf.sort_values(axis=0, ascending=False, inplace=True)

name_rf = list(feature_imp_series_rf.index)
rate_rf = list(feature_imp_series_rf.values)

importance_show = pd.DataFrame({'feature':name_rf, 'importance':rate_rf})

plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['figure.figsize'] = (10, 6)
plt.bar(range(len(feature_imp_rf)), feature_imp_rf, color='r')
plt.xticks(range(len(feature_imp_rf)), Xtrain_rf.columns, rotation='vertical')
plt.title('Feature Importance_'+t)
plt.savefig(path+'output_%s.png' %t, bbox_inches='tight')

### 감성점수와 순매수 비율 간 OLS

In [ ]:
# for i in range(20):
#   if i == 5:
#     continue

t = top20['종목명'][i]

data = pd.read_csv(path+"합체 데이터/%s_합체.csv" %t).drop(columns = ['Unnamed: 0'])
data.rename(columns = {'개인 순매수비율' : 'ind_buy', '기관 순매수비율': 'agn_buy', '외국인 순매수비율': 'fegn_buy'}, inplace = True)

C_before = data[data.Date <= '2020-03-19'][['Date', 'compound', 'posneg', 'ind_buy', 'agn_buy', 'fegn_buy']].set_index('Date').dropna()
C_after = data[data.Date > '2020-03-19'][['Date', 'compound', 'posneg', 'ind_buy', 'agn_buy', 'fegn_buy']].set_index('Date').dropna()

if record[t][0][:3] == 'pos': 
  x_before = C_before['posneg']
  x_after = C_after['posneg']
else:
  x_before = C_before['compound']

 x_after = C_after['compound']

y_before1 = C_before['ind_buy']
# y_before1 = C_before['개인 순매수비율']
y_after1 = C_after['ind_buy']
# y_after1 = C_after['개인 순매수비율']

y_before2 = C_before['agn_buy']
y_after2 = C_after['agn_buy']

y_before3 = C_before['fegn_buy']
y_after3 = C_after['fegn_buy']

In [ ]:
model = sm.OLS(y_before1, x_before) # y_before1~3 & x_before # y_after1~3 & x_after
result1 = model.fit()
print(result1.summary())

plt.rc('figure', figsize=(12, 7))
plt.text(0.01, 0.05, str(result1.summary()), {'fontsize': 12}, fontproperties = 'monospace')
plt.axis('off')
plt.tight_layout()
plt.savefig(path+'%s 개인_OLS(%s).png' %(t, "before Feb")) # "since Feb"